In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
%matplotlib inline

In [2]:
df = pd.read_csv('data/Train_Test_datasets/Train_Test_Network_dataset/Train_Test_Network.csv', skipinitialspace=True)

df.head()

,ts,src_ip,src_port,dst_ip,dst_port,proto,service,duration,src_bytes,dst_bytes,...,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,type
0,1554198358,3.122.49.24,1883,192.168.1.152,52976,tcp,-,80549.530260,1762852,41933215,...,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
1,1554198358,192.168.1.79,47260,192.168.1.255,15600,udp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,0,normal
2,1554198359,192.168.1.152,1880,192.168.1.152,51782,tcp,-,0.000000,0,0,...,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
3,1554198359,192.168.1.152,34296,192.168.1.152,10502,tcp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,0,normal
4,1554198362,192.168.1.152,46608,192.168.1.190,53,udp,dns,0.000549,0,298,...,0,0,-,-,-,bad_UDP_checksum,-,F,0,normal


In [3]:
df.drop(columns=['ts', 'src_ip', 'src_port', 'dst_ip', 'dst_port'], inplace=True)

# for now, only work with binary classification. ignore attack type, 'type'
df.drop(columns=['type'], inplace=True)

In [4]:
df.head()

,proto,service,duration,src_bytes,dst_bytes,conn_state,missed_bytes,src_pkts,src_ip_bytes,dst_pkts,...,http_request_body_len,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label
0,tcp,-,80549.530260,1762852,41933215,OTH,0,252181,14911156,2,...,0,0,0,-,-,-,bad_TCP_checksum,-,F,0
1,udp,-,0.000000,0,0,S0,0,1,63,0,...,0,0,0,-,-,-,-,-,-,0
2,tcp,-,0.000000,0,0,OTH,0,0,0,0,...,0,0,0,-,-,-,bad_TCP_checksum,-,F,0
3,tcp,-,0.000000,0,0,OTH,0,0,0,0,...,0,0,0,-,-,-,-,-,-,0
4,udp,dns,0.000549,0,298,SHR,0,0,0,2,...,0,0,0,-,-,-,bad_UDP_checksum,-,F,0


In [5]:
df.columns

Index(['proto', 'service', 'duration', 'src_bytes', 'dst_bytes', 'conn_state',
       'missed_bytes', 'src_pkts', 'src_ip_bytes', 'dst_pkts', 'dst_ip_bytes',
       'dns_query', 'dns_qclass', 'dns_qtype', 'dns_rcode', 'dns_AA', 'dns_RD',
       'dns_RA', 'dns_rejected', 'ssl_version', 'ssl_cipher', 'ssl_resumed',
       'ssl_established', 'ssl_subject', 'ssl_issuer', 'http_trans_depth',
       'http_method', 'http_uri', 'http_version', 'http_request_body_len',
       'http_response_body_len', 'http_status_code', 'http_user_agent',
       'http_orig_mime_types', 'http_resp_mime_types', 'weird_name',
       'weird_addl', 'weird_notice', 'label'],
      dtype='object')

In [6]:
df.dtypes

proto                      object
service                    object
duration                  float64
src_bytes                   int64
dst_bytes                   int64
conn_state                 object
missed_bytes                int64
src_pkts                    int64
src_ip_bytes                int64
dst_pkts                    int64
dst_ip_bytes                int64
dns_query                  object
dns_qclass                  int64
dns_qtype                   int64
dns_rcode                   int64
dns_AA                     object
dns_RD                     object
dns_RA                     object
dns_rejected               object
ssl_version                object
ssl_cipher                 object
ssl_resumed                object
ssl_established            object
ssl_subject                object
ssl_issuer                 object
http_trans_depth           object
http_method                object
http_uri                   object
http_version               object
http_request_b

In [7]:
df.weird_name.value_counts()

-                                   459749
DNS_RR_unknown_type                    753
active_connection_reuse                275
data_before_established                109
bad_UDP_checksum                        68
bad_TCP_checksum                        52
connection_originator_SYN_ack           20
above_hole_data_without_any_acks         6
inappropriate_FIN                        3
dnp3_corrupt_header_checksum             3
possible_split_routing                   3
TCP_ack_underflow_or_misorder            2
Name: weird_name, dtype: int64

In [8]:
categorical_columns = [
    'proto',
    'service',
    'conn_state',
    'dns_AA',
    'dns_RD',
    'dns_RA',
    'dns_rejected',
    'dns_query',
    'ssl_version',
    'ssl_cipher',
    'ssl_resumed',
    'ssl_established',
    'ssl_subject',
    'ssl_issuer',
    'http_trans_depth',
    'http_method',
    'http_uri',
    'http_version',
    'http_user_agent',
    'http_orig_mime_types',
    'http_resp_mime_types',
    'weird_name',
    'weird_addl',
    'weird_notice',
]
numerical_columns = set(df.columns) - set(categorical_columns)
numerical_columns

{'dns_qclass',
 'dns_qtype',
 'dns_rcode',
 'dst_bytes',
 'dst_ip_bytes',
 'dst_pkts',
 'duration',
 'http_request_body_len',
 'http_response_body_len',
 'http_status_code',
 'label',
 'missed_bytes',
 'src_bytes',
 'src_ip_bytes',
 'src_pkts'}

In [9]:
# total_unique_values = 0
# for col in categorical_columns:
#     num_unique_values = df[col].nunique()
#     print(col, '(', num_unique_values, "unique values )", '\n', df[col].value_counts(), "\n")
#     total_unique_values += num_unique_values

# print(total_unique_values, "unique categorical values")

In [10]:
# drop the dns_query column for now - it has 14372 unique values, so we can't really one-hot encode it
df.drop(columns=['dns_query'], inplace=True)

In [11]:
categorical_columns = list(set(categorical_columns) - set(['dns_query']))
print(categorical_columns)

['weird_notice', 'http_method', 'http_resp_mime_types', 'ssl_resumed', 'http_user_agent', 'ssl_cipher', 'ssl_version', 'http_orig_mime_types', 'weird_addl', 'dns_RD', 'proto', 'ssl_subject', 'service', 'weird_name', 'dns_rejected', 'http_trans_depth', 'dns_RA', 'conn_state', 'ssl_issuer', 'http_uri', 'dns_AA', 'http_version', 'ssl_established']


In [12]:
pd.get_dummies(df, columns=categorical_columns)

,duration,src_bytes,dst_bytes,missed_bytes,src_pkts,src_ip_bytes,dst_pkts,dst_ip_bytes,dns_qclass,dns_qtype,...,http_uri_/webmail/src/login.php,http_uri_/webupd8team/java/ubuntu/dists/bionic/InRelease,dns_AA_-,dns_AA_F,dns_AA_T,http_version_-,http_version_1.1,ssl_established_-,ssl_established_F,ssl_established_T
0,80549.530260,1762852,41933215,0,252181,14911156,2,236,0,0,...,0,0,1,0,0,1,0,1,0,0
1,0.000000,0,0,0,1,63,0,0,0,0,...,0,0,1,0,0,1,0,1,0,0
2,0.000000,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,1,0,0
3,0.000000,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,1,0,0
4,0.000549,0,298,0,0,0,2,354,0,0,...,0,0,1,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461038,0.000000,0,0,0,1,60,0,0,0,0,...,0,0,1,0,0,1,0,1,0,0
461039,0.000000,0,0,0,0,0,1,103,0,0,...,0,0,1,0,0,1,0,1,0,0
461040,290.371539,101568,2592,0,108,108064,31,3832,0,0,...,0,0,1,0,0,1,0,1,0,0
461041,23.190902,32,31,0,8,411,7,395,0,0,...,0,0,1,0,0,1,0,1,0,0


In [13]:
df[list(numerical_columns)]

,dns_qclass,http_status_code,dst_pkts,src_bytes,dns_rcode,duration,http_response_body_len,dns_qtype,src_ip_bytes,missed_bytes,dst_bytes,http_request_body_len,label,src_pkts,dst_ip_bytes
0,0,0,2,1762852,0,80549.530260,0,0,14911156,0,41933215,0,0,252181,236
1,0,0,0,0,0,0.000000,0,0,63,0,0,0,0,1,0
2,0,0,0,0,0,0.000000,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0.000000,0,0,0,0,0,0,0,0,0
4,0,0,2,0,0,0.000549,0,0,0,0,298,0,0,0,354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461038,0,0,0,0,0,0.000000,0,0,60,0,0,0,1,1,0
461039,0,0,1,0,0,0.000000,0,0,0,0,0,0,1,0,103
461040,0,0,31,101568,0,290.371539,0,0,108064,0,2592,0,1,108,3832
461041,0,0,7,32,0,23.190902,0,0,411,0,31,0,1,8,395


In [14]:
df_n = pd.get_dummies(df, columns=categorical_columns)

In [15]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [16]:
df_n[list(numerical_columns)] = pd.DataFrame(scaler.fit_transform(df[list(numerical_columns)]), columns=numerical_columns)

In [17]:
from sklearn.model_selection import train_test_split
y = df_n.label
x = df_n[list(set(df_n.columns) - set(['label']))]
# print(y.head())
# print(y.tail())
# print(x.columns)


In [30]:
%%time
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from time import time

# try 85/15 and 90/10
# make a graph to show accuracies

# tests = [.2, .15, .1]
test_size = .2
results = []

# for test_size in tests:
t1 = time()
clf = MLPClassifier(
    # hidden_layer_sizes=(1),
    # random_state=3
)
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    # random_state=2,
                                                    test_size=test_size
                                                   )
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
t2 = time()

result = {
    'test_size': test_size,
    'accuracy': accuracy_score(y_test, y_pred),
    'conf_matrix': confusion_matrix(y_test, y_pred),
    'time': t2 - t1,
    'f1_score': f1_score(y_test, y_pred)
}
print(result)

{'test_size': 0.2, 'accuracy': 0.8706959190534547, 'conf_matrix': array([[52538,  7456],
       [ 4467, 27748]]), 'time': 193.8762707710266, 'f1_score': 0.8231507438555897}
CPU times: user 12min 53s, sys: 25min 43s, total: 38min 37s
Wall time: 3min 13s


In [22]:
sum(y_test)

32393.0

In [24]:
len(y_test)

92209

In [25]:
sum([52323,  7493, 5717, 26676])

92209

In [27]:
sum([5717, 26676])

32393

In [29]:
sum(y_pred)

34169.0

In [20]:
print(x_test.columns)

Index(['http_trans_depth_9', 'conn_state_S0', 'http_uri_/?C=S;O=D',
       'ssl_resumed_F',
       'http_user_agent_Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML; like Gecko) Chrome/53.0.2785.143 Safari/537.36',
       'http_uri_/MFEwTzBNMEswSTAJBgUrDgMCGgUABBTBL0V27RVZ7LBduom/nYB45SPUEwQU5Z1ZMIJHWMys+ghUNoZ7OrUETfACEA8sEMlbBsCTf7jUSfg+hWk=',
       'http_uri_/msdownload/update/v3/static/trustedr/en/pinrulesstl.cab?b3d98506e52475df',
       'http_user_agent_Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:5.0) Whistler/20110021 myibrow/5.0.0.0',
       'http_user_agent_Comos/0.9_(robot@xyleme.com)', 'proto_udp',
       ...
       'http_uri_/RenderingControl_1.xml',
       'http_user_agent_Microsoft-Windows/10.0 UPnP/1.0 Microsoft-DLNA DLNADOC/1.50',
       'http_uri_/c/msdownload/update/others/2019/04/28699347_4732b1ab470d51de66dee45e4114cb6174f55279.cab',
       'conn_state_S1',
       'http_uri_/filestreamingservice/files/1f41d40e-6e49-42f0-a4dc-3d35029e2a5d?P1=1554315773&

In [21]:
for c in x_train.columns:
    print(c)

http_trans_depth_9
conn_state_S0
http_uri_/?C=S;O=D
ssl_resumed_F
http_user_agent_Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML; like Gecko) Chrome/53.0.2785.143 Safari/537.36
http_uri_/MFEwTzBNMEswSTAJBgUrDgMCGgUABBTBL0V27RVZ7LBduom/nYB45SPUEwQU5Z1ZMIJHWMys+ghUNoZ7OrUETfACEA8sEMlbBsCTf7jUSfg+hWk=
http_uri_/msdownload/update/v3/static/trustedr/en/pinrulesstl.cab?b3d98506e52475df
http_user_agent_Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:5.0) Whistler/20110021 myibrow/5.0.0.0
http_user_agent_Comos/0.9_(robot@xyleme.com)
proto_udp
http_uri_/icon_SML.png
http_uri_/ubuntu/dists/cosmic-updates/InRelease
ssl_subject_CN=*.wns.windows.com
src_pkts
conn_state_S2
http_uri_-
http_uri_/c/msdownload/update/others/2019/04/28698818_bfacd682bfb9bf10a7396fb5e080fa0a38d1b153.cab
http_uri_/filestreamingservice/files/0791715e-2de4-47f2-ba02-d980d28e51c3/pieceshash
ssl_version_TLSv13
http_uri_/d/msdownload/update/others/2019/04/28701422_19fc1c232bf7eb4d3ddd097afebcaa3a0479d76c.cab
http_uri